In [3]:
import os
import json
import random
import copy
# import google.generativeai as genai
from google import genai

# load the API key, use your own API key
with open('../API_KEY.txt', 'r') as f:
    key = f.read()

client = genai.Client(api_key=key)

In [4]:
# import the concept discs: A.json, B.json, C.json and prompt.json in with a helper function

import json

def import_concept_discs(path):
    # research theme
    with open(path + 'A.json', 'r') as f:
        As = json.load(f)

    # research domain
    with open(path + 'B.json', 'r') as f:
        Bs = json.load(f)

    # research architecture
    with open(path + 'C.json', 'r') as f:
        Cs = json.load(f)

    with open(path + 'prompt.json', 'r') as f:
        prompts = json.load(f)

    with open(path + 'temp.json', 'r') as f:
        temps = json.load(f)

    return As, Bs, Cs, prompts, temps

path = './resource/seed/'
As, Bs, Cs, prompts, temps = import_concept_discs(path)

print(As)
print(Bs)
print(Cs)
print(prompts)
print(temps)

[['adaptive'], ['less is more'], ['hierarchical'], ['scaling'], ['in-the-wild'], ['self-refine'], ['look-ahead'], ['hindsight'], ['memory'], ['self-'], ['rethink'], ['weak to strong'], ['granularity'], ['in-context learning'], ['reference free'], ['grokking']]
[['agent'], ['planning'], ['RAG'], ['safety'], ['calibration'], ['reasoning'], ['persuasion'], ['debate']]
[['mamba'], ['RL'], ['linear models'], ['KV'], ['quantization'], ['optimal transport'], ['diffusion'], ['self-attention'], ['self-supervision']]
['You are a senior professor in AI and your students propose to do <combination>.Can you refine the title into a good one that can be accepted by top conferences such as ACL/EMNLP/NeurIPS/ICLR, etc. and list the potential abstract, introduction outline, and section names? \n\n Requirements: 1. do not hallucinate, 2. do not use any existing paper names in your pretraining data. 3. make sure the title and outline is with an outstanding paper quality so that your student can be happy a

In [9]:
# load the gemini client and generate content

def gen_with_gemini(client, prompt):
    response = client.models.generate_content(
        model="gemini-1.5-pro",
        contents=prompt,
        config=genai.types.GenerateContentConfig(
            candidate_count=2,
            max_output_tokens=1600,
            temperature=0.7,
        )
    )

    print(response.candidates[0].content.parts[0].text)
    candidates = [cand.content.parts[0].text for cand in response.candidates]
    return candidates


# trial run with a prompt
prompt = "What is the capital of France?"
response = gen_with_gemini(client, prompt)
print(response)



Paris

['Paris\n', 'Paris\n']


In [ ]:
# Spinning up the gemini client

# safe save the response to folder named ./response/
# the response is a json object with the following keys:  combination, prompt, response
def save_response(response, folder_name):
    if not os.path.exists(folder_name):
        os.makedirs(folder_name)

    # if the response name exist, add a number to the end
    if os.path.exists(folder_name + '/response.txt'):
        i = 1
        while os.path.exists(folder_name + '/response_' + str(i) + '.txt'):
            i += 1
        folder_name = folder_name + '/response_' + str(i) + '.txt'
    else:
        folder_name = folder_name + '/response.txt'

    with open(folder_name, 'w') as f:
        json.dump(response, f)

def main(As, Bs, Cs, prompts, templates, gemini_client):

    # random sample one or two words from theme, domain, and architecture
    # read the template, count how many Ax, Bx, Cx in the template
    # replace the Ax, Bx, Cx with the randomly selected words from the disc
    this_template = copy.deepcopy(templates[0])

    mapping = {"A": As, "B": Bs, "C": Cs}
    for i in range(3):
        for keyword in ["A", "B", "C"]:
            this_word = keyword + str(i)
            if this_word in this_template:
                random.seed(2077+i)
                chosen = random.choice(mapping[keyword])
                this_template = this_template.replace(this_word, random.choice(chosen))

    print("Combined concepts in a template: ", this_template)

    # generate a prompt
    prompt = prompts[0].replace("<combination>", this_template)
    print("Prompt: ", prompt)

    # generate a response
    response = gen_with_gemini(gemini_client, prompt)

    folder_name = "./response/"
    output_json = {"combination": this_template, "template": templates[0], "prompt": prompt, "response": response}
    save_response(response, folder_name)

    return response

response = main(As, Bs, Cs, prompts, temps, client)
print(response)


Combined concepts in a template:  less is more, agent, mamba
Let's refine this title and outline to something more conference-ready.  The core idea of "less is more" with agents like Mamba needs to be unpacked into a concrete contribution.  We'll focus on efficiency and performance.

**Title Options (pick one based on the final angle of the paper):**

1. **Sparse Activation Agents: Achieving Efficient Performance with MAMBA** (Focuses on the mechanism)
2. **MAMBA:  A Minimalist Agent Architecture for Enhanced Sample Efficiency** (Focuses on efficiency)
3. **Towards Efficient Generalization:  Exploring Sparse Activation Policies in Reinforcement Learning** (Broader, focuses on the overall goal)


**Abstract (Example using Title 1):**

Large language models (LLMs) have demonstrated impressive capabilities as agents, but their computational cost can be prohibitive. We introduce MAMBA (Minimally Activated Model-Based Agent), a novel agent architecture designed to achieve high performance w